In [1]:
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score,accuracy_score

Using TensorFlow backend.
D:\Anaconda3\envs\Tensor\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\envs\Tensor\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\envs\Tensor\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\envs\Tensor\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
def func(x,A,T1,T2):
    return A*(np.exp(-x/T1) - np.exp(-x/T2))

In [3]:
def create_signals(num_samples,scale = 0,amp = 3000):
    size = 1024
    val_x = np.arange(1024)
    labels_signal = []
    signals = []
    for i in range(num_samples):
        if i%1000==0:
            print(i)
        A = np.random.randint(1,amp)
        T1 = np.random.randint(3,30)
        T2 = 2
        stride = np.random.randint(100,900)
        y = np.zeros(size)
        
        y[stride:] = func(val_x[:size-stride],A,T1,T2)
        y[:stride] = func(val_x[size-stride:],A,T1,T2)
        
        y = [np.abs(i + np.random.normal(scale=scale)+scale) for i in y]
        signals.append(y)
        labels_signal.append(1)
    
    for i in range(num_samples):        
        y = np.zeros(size)                
        y = [np.abs(np.random.normal(scale=scale)+scale) for _ in range(size)]
        signals.append(y)
        labels_signal.append(0)
    
    data = pd.DataFrame(signals)
    data['labels'] = labels_signal
    return data

In [4]:
num = 500
data = create_signals(num,scale = 1/100, amp = 1)
for amp in range(5,1005,10):
    data1 = create_signals(num,scale = amp/100, amp = amp)
    data  = data.merge(data1,how = 'outer')
data = data.sample(frac = 1).reset_index(drop=True)
#data = pd.read_csv("Signals.csv")

In [5]:
data.shape

(101000, 1025)

In [9]:
size = 1024
input_layer = keras.layers.Input(shape = (None,size), name = 'Input_Signal')
x = keras.layers.LSTM(1024,return_sequences=True, name = 'LSTM_1',activation='relu')(input_layer)
x = keras.layers.LSTM(1024,return_sequences=True, name = 'LSTM_2',activation='relu')(x)
x = keras.layers.LSTM(1024,return_sequences=True, name = 'LSTM_3',activation='relu')(x)
x = keras.layers.LSTM(1024, name = 'LSTM_4',activation='relu')(x)        
output_layer = keras.layers.Dense(1,activation = 'sigmoid', name = 'Output_Answer')(x)
model = keras.models.Model(input_layer,output_layer)
model.compile(loss = keras.losses.binary_crossentropy,
              optimizer = keras.optimizers.RMSprop(),
              metrics = ['acc'])

In [10]:
X_train = data.iloc[:int(data.shape[0]*0.8),:-1]
y_train = data.iloc[:int(data.shape[0]*0.8),-1:]['labels']

X_test = data.iloc[int(data.shape[0]*0.8):,:-1]
y_test = data.iloc[int(data.shape[0]*0.8):,-1:]['labels']

X_train = np.array(X_train)
X_test = np.array(X_test)

X_train_mn = X_train/np.array(data.iloc[:,:-1]).max()

X_test_mn = X_test/np.array(data.iloc[:,:-1]).max()

X_train_mn = X_train_mn.reshape(X_train_mn.shape[0],1,X_train_mn.shape[1])
X_test_mn = X_test_mn.reshape(X_test_mn.shape[0],1,X_test_mn.shape[1])

X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])

In [11]:
er = keras.callbacks.EarlyStopping(monitor = 'val_acc',patience=3) 
mc = keras.callbacks.ModelCheckpoint('Model.h5',monitor='val_acc',save_best_only=True)

model.fit(X_train_mn,y_train,validation_split=0.2,batch_size=512,epochs = 100,callbacks=[er,mc])
y_pred = model.predict(X_test_mn)
y_pred = [1 if i>0.6 else 0 for i in y_pred]
f1_score(y_test,y_pred)

Train on 64640 samples, validate on 16160 samples
Epoch 1/100
64640/64640 [==============================] - 182s 3ms/step - loss: 0.4189 - acc: 0.7916 - val_loss: 0.3671 - val_acc: 0.8326
Epoch 2/100
64640/64640 [==============================] - 151s 2ms/step - loss: 0.3050 - acc: 0.8767 - val_loss: 0.2499 - val_acc: 0.8978
Epoch 3/100
64640/64640 [==============================] - 151s 2ms/step - loss: 0.2736 - acc: 0.8938 - val_loss: 0.2060 - val_acc: 0.9363
Epoch 4/100
64640/64640 [==============================] - 151s 2ms/step - loss: 0.2547 - acc: 0.9048 - val_loss: 0.2263 - val_acc: 0.9139
Epoch 5/100
64640/64640 [==============================] - 151s 2ms/step - loss: 0.2427 - acc: 0.9116 - val_loss: 0.2467 - val_acc: 0.9058
Epoch 6/100
64640/64640 [==============================] - 150s 2ms/step - loss: 0.2309 - acc: 0.9157 - val_loss: 0.4466 - val_acc: 0.7815
Epoch 7/100
64640/64640 [==============================] - 151s 2ms/step - loss: 0.2279 - acc: 0.9195 - val_loss: 0.

0.9669924773553042

In [20]:
from keras.utils.vis_utils import plot_model
model = keras.models.load_model("Model.h5")

In [87]:
y_pred = model.predict(X_test)
y_pred = [1 if i>0.6 else 0 for i in y_pred]
print(f1_score(y_test,y_pred,average = "macro"))
accuracy_score(y_test,y_pred)

0.9818413507486461


0.9821875